# MD3 Modeļa pielāgošana

### Autors: Agris Pudāns, ap08426.

Uzdevuma apraksts
### Uzdevuma apraksts

Jāizveido modelis, kas spēj klasificēt Steam spēļu atsauksmes (angļu valodā) starp rekomendējošām un nerekomendējošām, izmantojot priekšapmācītu DistilBERT modeli. 

**Datu struktūra**:
- **app_id**: Spēles unikālais Steam ID
- **app_name**: Spēles nosaukums 
- **review_text**: Atsauksmes teksts (klasifikācijas pamats)
- **review_score**: Spēles novērtējums: 1 - pozitīvs, -1 - negatīvs
- **review_votes**: Citu spēlētāju balsis par atsauksmes lietderīgumu

## Kontroles bloks
Definē slēdzi un dažādas konstantes

In [24]:
# True - ielādē treniņa datu kopu un uz tās apmāca jaunu modeli, kuru saglabā
# False - ielādē jau apmācītu modeli un atsevišķu datu kopu, uz kuras modeli pārbaudīt
TRAIN = True

# Treniņa datu kopas ielādes URL. Kaggle datu atrašanās vieta.
TRAIN_DATA_URL = "/kaggle/input/steam-reviews/dataset_top20_cleaned.csv"

# Apmācīta modeļa ielādes URL.
MODEL_URL = "steam_reviews_model.pt"


# Globālie mainīgie
model = None
tokenizer = None

## Moduļu importēšana

In [23]:
# os modulis nepieciešams darbam ar failiem
import os
# torch modulis nodrošina MI apmācībai un lietošanai nepieciešamās funkcijas
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
# numpy modulis nodrošina dažādas matemātiskās funkcijas
import numpy as np
# Pandas modulis tabulveida datu apstrādei
import pandas as pd
# requests modulis tīkla pieprasījumiem
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Hugging Face Transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import logging
logging.set_verbosity_error()  # Samazinām transformers bibliotēkas ziņojumu daudzumu

# Novērtēšanas metrikas
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Vienmēr cpu Kagglē
device = "cpu"

## Klašu un funkciju definēšana

Visas klases un funkcijas, kuras jāizmanto uzdevumam.

#### Klase SteamReviewsDataset, kas saturēs datus neironu tīklam saprotamā formātā

In [10]:
class SteamReviewsDataset(Dataset):
    def __init__(self, texts, scores, tokenizer, max_length=128):
        self.reviews = reviews
        self.scores = scores
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        score = self.scores[idx]
        
        # Tokenizējam atsauksmes tekstu
        encoding = self.tokenizer(
            review,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Pārveidojam vērtējumu atbilstoši DistilBERT formātam (0 vai 1), kur +1 ir OK, 0 nav OK.
        label = 1 if score == 1 else 0
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

#### Klase ReviewClassifier, kas ir neironu tīkla klasifikators

In [25]:
class ReviewClassifier:
    def __init__(self):
        # Ielādē priekšapmācīto DistilBERT modeli un tokenizeri
        self.model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
        self.tokenizer = DistilBertTokenizer.from_pretrained(self.model_name)
        self.model = DistilBertForSequenceClassification.from_pretrained(self.model_name)
        self.model.to(device)
    
    def get_tokenizer(self):
        return self.tokenizer
    
    def get_model(self):
        return self.model
    
    def save_model(self, path="steam_reviews_model.pt"):
        torch.save(self.model.state_dict(), path)
        print(f"Modelis saglabāts: {path}")
    
    def load_model(self, path):
        try:
            if path.startswith('http'):
                import requests
                from io import BytesIO
                response = requests.get(path)
                if response.status_code == 200:
                    weights = BytesIO(response.content)
                    self.model.load_state_dict(torch.load(weights, map_location=device))
                    print(f"Modelis ielādēts no URL: {path}")
                else:
                    print(f"Kļūda ielādējot modeli no URL: {response.status_code}")
                    return False
            else:
                self.model.load_state_dict(torch.load(path, map_location=device))
                print(f"Modelis ielādēts no: {path}")
            return True
        except Exception as e:
            print(f"Kļūda ielādējot modeli: {str(e)}")
            return False
    
    def predict(self, texts):
        # Pārslēdz modeli uz prognozēšanu
        self.model.eval()
        predictions = []
        
        # Atmiņas optimizēšanai apstrādā datus pa grupām
        batch_size = 32
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenizē tekstu
            inputs = self.tokenizer(
                batch_texts,
                max_length=MAX_LENGTH,
                padding='max_length',
                truncation=True,
                return_tensors="pt"
            ).to(device)
            
            # Veic prognozes bez gradienta
            with torch.no_grad():
                outputs = self.model(**inputs)
                logits = outputs.logits
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                
                # Konvertē atpakaļ no DistilBert uz Steam skalu, kur +1 ir OK, un -1 nav OK
                steam_preds = [1 if p == 1 else -1 for p in preds]
                predictions.extend(steam_preds)
        
        return predictions

#### load_and_prepare_data - ielādē un sagatavo datus

In [33]:
def load_and_prepare_data(file_path):
    print(f"Ielādē datus no: {file_path}")
    
    # Ielādē datus
    try:
        # data = pd.read_csv(file_path)
        data = pd.read_csv(data_url, error_bad_lines=False)  # Ignorē problemātiskās rindas
        print(f"Kopējais ierakstu skaits: {len(data)}")
    except Exception as e:
        print(f"Kļūda ielādējot datus: {e}")
        return None
    
    # Attīra tekstu
    data['clean_review'] = data['review_text'].apply(lambda x: re.sub(r'[^\x00-\x7F]+', ' ', str(x)))
    
    # Izdrukā klašu sadalījumu
    class_counts = data['review_score'].value_counts()
    print(f"Klašu sadalījums: {class_counts.to_dict()}")
    
    # Iegūst  atsauksmes un novērtējumus
    review_texts = data['clean_review'].tolist()
    review_scores = data['review_score'].tolist()

#### train - treniņš ar apmācības datu kopu

In [27]:
def train():
    print("Apmāca modeli...")
    global model, tokenizer
    
    # Ielādē datus
    df, reviews, scores = load_and_prepare_data(TRAIN_DATA_URL)
    
    # Inicializējam modeli un iegūstam tokenizētāju
    classifier = ReviewClassifier()
    model = classifier.get_model()
    tokenizer = classifier.get_tokenizer()
    
    # Izveido datu kopas
    train_dataset = SteamReviewDataset(reviews, scores, tokenizer)

    # Izveido datu ielādētājus
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Definē optimizatoru
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    
    # Apmāca modeli
    model.train()
    
    for epoch in range(EPOCHS):
        print(f"Epohā {epoch+1}/{EPOCHS}")
        total_loss = 0
        
        for batch in train_dataloader:
            # Pārvieto batch uz ierīci (cpu)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Notīra vecos gradientus
            optimizer.zero_grad()
            
            # Izpilda modeli
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            
            # Atjaunina parametrus
            optimizer.step()
            total_loss += loss.item()
        
        # Izvada vidējo zaudējumu
        avg_loss = total_loss / len(train_dataloader)
        print(f"Vidējais zaudējums: {avg_loss:.4f}")
    
    print("Apmācība pabeigta!")
    return classifier



#### test - pārbauda modeli ar iedoto datu kopu

In [28]:
def test():
    print("Testē modeli...")
    
    global model, tokenizer
    
    # Ielādē un sagatavojam datus
    df, reviews, scores = prepare_data(TEST_DATA_URL)
    print(f"Ielādēti {len(reviews)} ieraksti testēšanai")
    
    # Inicializē klasifikatoru
    if model is None or tokenizer is None:
        classifier = ReviewClassifier()
        tokenizer = classifier.get_tokenizer()
        model = classifier.get_model()
    else:
        classifier = ReviewClassifier()
        classifier.model = model
        classifier.tokenizer = tokenizer
    
    # Veic prognozes
    predictions = classifier.predict(reviews)
    
    # Aprēķina precizitāti
    correct = 0
    total = len(predictions)
    
    # Izvada rezultātus
    for i in range(total):
        real_score = scores[i]
        pred_score = predictions[i]
        result = "pareizi" if real_score == pred_score else "nepareizi"
        
        text = reviews[i]
        # Saīsinām garus tekstus izvadei
        if len(text) > 50:
            text = text[:47] + "..."
        
        print(f"{i+1}. datu rindiņā klasifikatora rezultāts: {pred_score}; reālais novērtējums: {real_score}, {result}")
        print(f"   Atsauksmes teksts: {text}")
        
        if real_score == pred_score:
            correct += 1
    
    # Izvada kopējo precizitāti
    accuracy = (correct / total) * 100
    print(f"Precizitāte: {accuracy:.2f}% ({correct}/{total})")

#### saveWeights - saglabāt modeļa svarus

In [29]:
def saveWeights():
    """
    Saglabā modeļa svarus
    """
    print("Saglabā modeļa svarus...")
    global model
    
    # Pārbaudām, vai modelis ir ielādēts
    if model is None:
        print("Kļūda: Modelis nav inicializēts!")
        return
    
    # Izveidojam klasifikatoru
    classifier = ReviewClassifier()
    classifier.model = model
    
    # Saglabājam modeli
    classifier.save_model("steam_reviews_model.pt")
    
    print("Modelis ir saglabāts! Augšupielādējiet to uz publisko serveri un atjauniniet MODEL_URL vērtību.")


loadWeights - ielasa modeļa svarus

In [30]:
def loadWeights():
    """
    Ielādē modeļa svarus no MODEL_URL
    """
    print(f"Ielādē modeļa svarus no {MODEL_URL}...")
    global model, tokenizer
    
    # Izveidojam klasifikatoru
    classifier = ReviewClassifier()
    tokenizer = classifier.get_tokenizer()
    model = classifier.get_model()
    
    # Mēģinām ielādēt svarus
    success = classifier.load_model(MODEL_URL)
    
    if success:
        print("Modeļa svari veiksmīgi ielādēti!")
    else:
        print("Turpinām ar bāzes modeli...")

#### show_data_samples - parādam (5) datu paraugus

In [31]:
def show_data_samples(data_url, n_samples=5):
    """
    Parāda datu paraugus
    
    Parametri:
    data_url (str): Ceļš uz datu failu
    n_samples (int): Cik paraugus parādīt
    """
    try:
        data = pd.read_csv(data_url)
        print(f"Datu struktūra ({len(data)} ieraksti):")
        print(data.columns.tolist())
        
        print(f"\nPirmie {n_samples} ieraksti:")
        for i, row in data.head(n_samples).iterrows():
            print(f"Ieraksts #{i+1}:")
            print(f"  Spēle: {row.get('app_name', 'N/A')} (ID: {row.get('app_id', 'N/A')})")
            print(f"  Novērtējums: {row.get('review_score', 'N/A')}")
            print(f"  Balsis: {row.get('review_votes', 'N/A')}")
            print(f"  Atsauksme: {row.get('review_text', 'N/A')[:100]}...")
            print()
    except Exception as e:
        print(f"Kļūda ielādējot datus: {e}")

## Datu ielāde

Notiek datu ielāde no norādītas datu kopas 

In [35]:
if TRAIN:
    print("Ielādējam treniņa datu paraugus:")
    show_data_samples(TRAIN_DATA_URL, n_samples=5)
else:
    print("Ielādējam testa datu paraugus:")
    show_data_samples(TEST_DATA_URL, n_samples=5)

NameError: name 'j' is not defined

## Modeļa apmācīšana

Izpilda, ja TRAIN == true

In [34]:
if TRAIN:
    train()

Apmāca modeli...
Ielādē datus no: /kaggle/input/steam-reviews/dataset_top20_cleaned.csv
Kļūda ielādējot datus: name 'data_url' is not defined


TypeError: cannot unpack non-iterable NoneType object

## Saglabā datus

Saglabā tikai, ja TRAIN == true

In [ ]:
if TRAIN:
    saveWeights()

## Modeļa testēšana

In [20]:
test()

Testē modeli...


NameError: name 'model' is not defined